In [ ]:
#| default_exp production.metadata

In [ ]:
# | export


from p4tools.production import projection


import logging
from planetarypy.pds.apps import get_index
from planetarypy.hirise import ProductPathfinder
import pandas as pd

In [ ]:
# | export

logger = logging.getLogger(__name__)

In [ ]:
class MetadataReader:

    def __init__(self, obsid):
        self.obsid = obsid
        self.prodid = ProductPathfinder(obsid+"_COLOR")
        if not self.labelpath.exists():
            self.download_label()

    def read_edr_index(self):
        return get_index("mro.hirise", "edr")

    @property
    def labelpath(self):
        return self.prodid.local_label_path

    def download_label(self):
        self.prodid.download_label()

    @property
    def label(self):
        return labels.HiRISE_Label(self.labelpath)

    @property
    def campt_out_path(self):
        p4m = projection.P4Mosaic(self.obsid)
        return p4m.mosaic_path.parent / f"{self.obsid}_campt_out.csv"

    @property
    def campt_out_df(self):
        return pd.read_csv(self.campt_out_path)


In [ ]:
# | export

def get_north_azimuths_from_SPICE(obsids):
    NAs = []
    for obsid in obsids:
        meta = MetadataReader(obsid)
        NAs.append(meta.campt_out_df['NorthAzimuth'].median())
    return pd.DataFrame(dict(OBSERVATION_ID=obsids, north_azimuth=NAs))